In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
from collections import Counter
import urllib.request
import zipfile
import os

# Configure device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(f"Using device: {device}")

if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"  # For Apple Silicon
else:
    device = "cpu"

device = torch.device(device)

print("using device ",device)

# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Store results for comparison
results = {}

using device  mps
